In [ ]:
import mysql.connector as con
from mysql.connector import Error
import pandas as pd
import numpy as np
import matplotlib as mb
import matplotlib.pyplot as plt
import seaborn as sb
%matplotlib inline

In [ ]:
#pip install mysql-connector-python-rf

In [ ]:
def database_connection(userId,pwd,hostname,dbinfo):
   cnx = con.connect(user=userId,passwd=pwd,host=hostname,database=dbinfo)
   return cnx

def fetch_columns(tablename,connection):
   colnames = list()
   datatypes = list()
   cursor = connection.cursor()
   query="desc " + tablename;
   cursor.execute(query)
   record = cursor.fetchone()
   while record != None:
      colnames.append(record[0])
      datatypes.append(record[1])
      record = cursor.fetchone()
   cursor.close()
   return colnames,datatypes

def fetch_records(tablename,connection):
   cursor = connection.cursor()
   query="select * from " + tablename
   cursor.execute(query)
   records = list(cursor.fetchall())
   cursor.close()
   return records
   

In [ ]:
dataF = {}
def ExtractData(tablename,connection):
   columnNames = list()
   dataTypes = list()
   dataList = list()
   columnNames, dataTypes = fetch_columns(tablename, connection)
   dataList = fetch_records(tablename,connection)
   df = pd.DataFrame(data=dataList,columns=columnNames)
   print(columnNames)
   return df
   
def EstablishConnection(userId,pwd,host,db):   
   connection = database_connection(userId,pwd,host,db)
   try:
      if connection.is_connected():
         number = 0
         for table in TableList:
            dataF[number] = ExtractData(table,connection)
            number = number + 1
   except Error as e:
      print("ERROR while connection to Mysql", e)
   finally:
      if connection.is_connected():
         connection.close()
   return

TableList = ['gender_age_train','phone_brand_device_model']
EstablishConnection('student','student','cpanel.insaid.co','Capstone1')

In [ ]:
eventsData = pd.read_csv('events_data_1012.csv')

In [ ]:
#pip install py-translate

In [ ]:
eventsData.isnull().sum()

In [ ]:
cityDict = dict()
cityDict = eventsData.set_index('city')['state'].to_dict()
eventsData['state'].fillna(eventsData.city.map(cityDict),inplace=True)
eventsData

In [ ]:
longDict = eventsData.set_index('city')['longitude'].to_dict()
latDict = eventsData.set_index('city')['latitude'].to_dict()
eventsData['longitude'].fillna(eventsData.city.map(longDict),inplace=True)
eventsData['latitude'].fillna(eventsData.city.map(latDict),inplace=True)

In [ ]:
latDeviceDict = eventsData.set_index('latitude')['device_id'].to_dict()
eventsData['device_id'].fillna(eventsData.latitude.map(latDeviceDict), inplace=True)
eventsData[eventsData['device_id'].isna()]

In [ ]:
eventsData.isnull().sum()

In [ ]:
#!pip install langdetect
eventsData.info()

In [ ]:
dataF[0]['device_id'] = dataF[0]['device_id'].abs()
dataF[1]['device_id'] = dataF[1]['device_id'].abs()

In [ ]:
df = pd.DataFrame()
df = pd.merge(dataF[0], dataF[1], how='inner', on='device_id')
df

In [ ]:
len(df['phone_brand'].unique())

In [ ]:
eventsData['device_id'] = eventsData['device_id'].abs()

In [ ]:
eventsMerged = pd.merge(eventsData,df,how='inner',on='device_id')
eventsMerged

In [ ]:
phoneBrandDict = dict()
phoneBrandDict = {'华为':'Huawei', '小米':'Xiaomi','三星':'Samsung','魅族':'Meizu','酷派':'Coolpad','乐视':'LeEco', '联想':'Lenovo'}
eventsMerged['phone_brand'].replace(phoneBrandDict,inplace=True)
eventsMerged

In [ ]:
import folium
from IPython.display import display

In [ ]:
tooltip = 'Click me!'
m = folium.Map(location=[28.730140,77.225676], zoom_start=5)
m.save('index.html')

In [ ]:
latLongCityList = eventsMerged.groupby(['latitude','longitude'])['city']
#.append(eventsData['city']).unique()
latLongdf = latLongCityList.first().to_frame(name="city")
latLongdf = latLongdf.reset_index()
latLongdf

In [ ]:
for each in latLongdf.iterrows():
    folium.Marker(location=[each[1]['latitude'],each[1]['longitude']], popup=each[1]['city']).add_to(m)

m.save('index.html')

In [ ]:
from  geopy.geocoders import Nominatim

In [ ]:
geolocator = Nominatim()
city="Indore"
country="India"
loc = geolocator.geocode(city+','+ country)
print("latitude is :-" ,loc.latitude,"\nlongtitude is:-" ,loc.longitude)

In [ ]:
latLongdf[latLongdf['city']=='Indore']

In [ ]:
latLongdf[((latLongdf['latitude'] < 22.0) | (latLongdf['latitude'] > 23.0)) & (latLongdf['city'] == "Indore")] 

In [ ]:
latLongCityCount = pd.DataFrame({'count' : eventsMerged.groupby(['latitude','longitude','city'])['city'].size()}).reset_index()
latLongCityCount[latLongCityCount['city'] == "Indore"]

In [ ]:
lat = latLongCityCount[latLongCityCount['count'] == latLongCityCount[latLongCityCount['city'] == "Indore"]['count'].max()]['latitude']
lon = latLongCityCount[latLongCityCount['count'] == latLongCityCount[latLongCityCount['city'] == "Indore"]['count'].max()]['longitude']

In [ ]:
latList = list(latLongdf[((latLongdf['latitude'] < 22.0) | (latLongdf['latitude'] > 23.0)) & (latLongdf['city'] == "Indore")]['latitude'])
lonList = list(latLongdf[((latLongdf['latitude'] < 22.0) | (latLongdf['latitude'] > 23.0)) & (latLongdf['city'] == "Indore")]['longitude'])

In [ ]:
print(latList)
print(lonList)

In [ ]:
#eventsMerged['latitude'] = eventsMerged['latitude'].astype(str)
#eventsMerged['longitude'] = eventsMerged['longitude'].astype(str)

In [ ]:
for l1,l2 in zip(latList,lonList):
    for i in eventsMerged[eventsMerged['city'] == "Indore"]['latitude'].loc[eventsMerged['latitude'] == l1].index.values.tolist():
        eventsMerged.at[i,'latitude'] = lat.values[0]
        eventsMerged.at[i,'longitude'] = lon.values[0]

In [ ]:
#eventsMerged['latitude'] = eventsMerged['latitude'].astype('float64')
#eventsMerged['longitude'] = eventsMerged['longitude'].astype('float64')

In [ ]:
eventsMerged[((eventsMerged['latitude'] < 22.0) | (eventsMerged['latitude'] > 23.0)) & (eventsMerged['city'] == "Indore")] 

In [ ]:
for l1,l2 in zip(latList,lonList):
    for i in latLongdf[latLongdf['city'] == "Indore"]['latitude'].loc[latLongdf['latitude'] == l1].index.values.tolist():
        latLongdf.at[i,'latitude'] = lat.values[0]
        latLongdf.at[i,'longitude'] = lon.values[0]

In [ ]:
for each in latLongdf.iterrows():
    folium.Marker(location=[each[1]['latitude'],each[1]['longitude']], popup=each[1]['city']).add_to(m)

m.save('index.html')

In [ ]:
eventsMerged['state'].unique()

In [ ]:
plt.figure(figsize=(10,10))
sb.countplot(x="state",data=eventsMerged)
plt.xticks(rotation=90)
plt.title("India Province")
plt.xlabel("State")
plt.ylabel("Count")
plt.show()

In [ ]:
phoneBranddf = pd.DataFrame({'count' : eventsMerged.groupby('phone_brand').size()}).reset_index()
phoneBranddf = phoneBranddf.sort_values('count',ascending=False).head(10)
phoneBranddf

In [ ]:
phoneBranddf['phone_brand'].values.tolist()

In [ ]:
plt.figure(figsize=(10,10))
sb.barplot(x="phone_brand",y="count",data=phoneBranddf)
plt.xticks(rotation=90)
plt.title("Brand")
plt.xlabel("Phone Brand")
#plt.axes(xticklabels=phoneBranddf['phone_brand'].values.tolist(), rotation=90)
plt.ylabel("Count")
plt.show()

In [ ]:
sb.countplot(x='gender',data=eventsMerged)
plt.title("Gender Distribution")
plt.xlabel("Gender")
#plt.axes(xticklabels=phoneBranddf['phone_brand'].values.tolist(), rotation=90)
plt.ylabel("Count")
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
sb.countplot(x='group',data=eventsMerged)
plt.xticks(rotation=90)
plt.title("Distrribution of Age Segements")
plt.xlabel("Age group")
plt.ylabel("Count")
plt.show()

In [ ]:
#sb.FacetGrid()
#ventsMerged.groupby(['phone_brand','group','state','gender'])
#ventsMerged.groupby

In [ ]:
TopBranddf = eventsMerged[eventsMerged.phone_brand.isin(phoneBranddf['phone_brand'].values.tolist())]
TopBranddf

In [ ]:
TopBranddf.drop_duplicates('device_id')

In [ ]:
plt.figure(figsize=(100,100), dpi= 80)
sb.set(style="ticks", color_codes=True)
g = sb.FacetGrid(data=TopBranddf, row='phone_brand', col='state', hue='group',margin_titles=True, aspect=10)
g = g.map(plt.hist,'gender')
plt.show()

In [ ]:
sb.catplot(data=TopBranddf, row='state', col='group',hue='gender', x= 'phone_brand', kind='count', height=2.5, aspect=3)

In [ ]:
sb.catplot(data=TopBranddf, row='state', col='phone_brand',hue='group', x= 'gender', kind='count', height=2.5, aspect=3)

In [ ]:
sb.catplot(data=TopBranddf, row='gender', col='phone_brand',hue='state', x= 'group', kind='count', height=2.5, aspect=3)

In [ ]:
len(TopBranddf['device_id'].unique())

In [ ]:
m = folium.Map(location=[48, -102], zoom_start=3)

folium.Choropleth(name='choropleth', data=eventsMerged, columns=['state','device_id'], legend_name='Customer Distribution in India').add_to(m)

m

In [ ]:
eventsMerged.timestamp = eventsMerged.timestamp.astype('datetime64')

In [ ]:
eventsMerged.info()

In [ ]:
timedf = eventsMerged.set_index('timestamp')
timedf['Year'] = timedf.index.year
timedf['Hour'] = timedf.index.hour
timedf['Month'] = timedf.index.month
timedf['Day'] = timedf.index.day
timedf['Weekday Name'] = timedf.index.weekday_name

In [ ]:
timedf = timedf.reset_index()

In [ ]:
timedf

In [ ]:
sb.boxplot(x='Hour',data=timedf)
plt.title("Hourly Distrribution of Phone calls")
plt.xlabel("Age group")
plt.ylabel("Count")
plt.show()